In [ ]:
!pip install -qU \
  pinecone-client==3.0.0 \
  pinecone-datasets==0.7.0 \
  sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.9/199.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
retriever = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2",device=device)
retriever

cpu


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [ ]:
from pinecone import Pinecone,PodSpec

# connect to pinecone environment
pc=Pinecone(
    api_key="95e66ee2-5711-41b7-8655-00761c0b3729",
)
pc.list_indexes()

{'indexes': [{'dimension': 768,
              'host': 'nepali-healthqa-lpqyynm.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'nepali-healthqa',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
query = "म धेरै थकित महसुस गर्छु र मेरो नाक बगिरहेको छ। साथै, मलाई घाँटी दुखेको छ र अलि टाउको दुखेको छ। मलाई के भइरहेको छ?"
query = retriever.encode(query)
index_name = "nepali-healthqa"
index = pc.Index(index_name)
results = index.query(vector=query.tolist(), top_k=2,include_metadata=True) #include_values=True
# results
answer = results['matches'][0]['metadata']
print(answer)


{'Answer': 'चिसो लक्षणहरूमा अक्सर लगातार हाच्छ्युँ गर्नु र नाक बग्नु समावेश छ। राम्रो महसुस गर्न:\nपानी, सूप र तातो चियाले हाइड्रेटेड रहनुहोस्।\nआराम गर्नुहोस् र पर्याप्त सुत्नुहोस्।\nरुघाखोकीको औषधि लिनुहोस्', 'Question': 'मलाई यो निरन्तर हाछ्युँ र नाक बगिरहेको छ। के यो सामान्य चिसो हुन सक्छ, कुनै सल्लाह?'}


In [ ]:
query = "मलाई कोविड लागेको छ, अलि अलि ज्वोरो पनि आएको छ,टाउको दुख्ने समस्या पनि रहेको छ । शितामोल खाएको सन्चो भएको छैन अझै । "
query = retriever.encode(query)
index_name = "nepali-healthqa"
index = pc.Index(index_name)

results = index.query(vector=query.tolist(), top_k=5,include_metadata=True) #include_values=True
print(results)
answer = results['matches'][0]['metadata']['Answer']
print(answer)


{'matches': [{'id': '4890',
              'metadata': {'Answer': '– बसेको अवस्थामा पनि चक्कर लाग्ने समस्या '
                                     'हुन्छ ।\n'
                                     '- यो विभिन्न चीजहरूको कारणले हुन सक्छ वा '
                                     'यो आफैमा टाढा हुन सक्छ।\n'
                                     '- के तपाईलाई चक्कर लाग्ने अन्य लक्षणहरू '
                                     'छन्?\n'
                                     '- यदि चक्कर आनाले तपाईको दैनिक जीवनमा '
                                     'असर गरिरहेको छैन भने, तपाइँ केहि दिन '
                                     'पर्खन सक्नुहुन्छ।\n'
                                     '- आफ्नो ख्याल राख्नुहोस् र '
                                     'नआत्तिनुहोस्।\n'
                                     '- यदि समस्या जारी रह्यो भने, सकेसम्म '
                                     'चाँडो चिकित्सकलाई देखाउनुहोस्।\n'
                                     '- डाक्टरले चक्कर लाग्नुको कारण पत्ता '
        

{'Answer': 'चिसो लक्षणहरूमा अक्सर लगातार हाच्छ्युँ गर्नु र नाक बग्नु समावेश छ। राम्रो महसुस गर्न:\nपानी, सूप र तातो चियाले हाइड्रेटेड रहनुहोस्।\nआराम गर्नुहोस् र पर्याप्त सुत्नुहोस्।\nरुघाखोकीको औषधि लिनुहोस्', 'Question': 'मलाई यो निरन्तर हाछ्युँ र नाक बगिरहेको छ। के यो सामान्य चिसो हुन सक्छ, कुनै सल्लाह?'}
